***Welcome! Hello!***

We are glad you're here 👏! And are excited you're into the tough stuff 💪!

This notebook exists to enable you to analyze a sequence of seed-key exchanges in a pcap containing CAN frames.

What follows builds upon the things done in the `easy_*.ipynb` and `medium_*.ipynb`; if you haven't gone through those yet, do so first.

In [ ]:
import getopt
import sys
import signal
import re
import threading
import time

from IPython.core.interactiveshell import InteractiveShell
from scapy.contrib.automotive.scanner.enumerator import ServiceEnumerator

InteractiveShell.ast_node_interactivity = 'all'

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import binascii
import pandas as pd
import qgrid

qgrid.enable()

from bokeh.plotting import figure, show
import bokeh.io

bokeh.io.reset_output()
bokeh.io.output_notebook()

In [ ]:
from scapy.all import *

load_layer("can")
conf.contribs['CANSocket'] = {'use-python-can': True}
load_contrib('cansocket')
load_contrib('isotp')
load_contrib('automotive.uds')
load_contrib('automotive.uds_scan')

# Do the thing: analyze seed key pairs

You need to already have a pcap containing CAN frames with seed key exchanges, lost of them. Maybe you automated button pushes in a diagnostic tool GUI to create a large repeated sequence of DSC, DSC_PR, SA, SA_PR, SA, SA_PR UDS messages? IDK, I'm not your boss.

Put the path to that pcap in the box below and push the 'Run Interact' ; you should get a table of all the diagnostic session levels, security levels, seeds and keys exchanged. If you have more than a single diagnostics session level or more than a single security level the rest of the notebook is not going to give you good results.

In [ ]:
def dataframe_of_seedkey_exchanges(msgs):
    df = pd.DataFrame()
    logger = logging.getLogger('dataframe_of_seedkey_exchanges')

    diag_level = None
    security_level = None
    seed = None

    diag_level_bytes = None
    security_level_bytes = None
    key_bytes = None

    for msg in msgs:
        if bytes(msg)[0] == 0x10 and len(msg) == 2:
            diag_level_bytes = bytes(msg)[1:]
            logger.debug('diag session requested level: ' + diag_level_bytes.hex())
            diag_level = None
            security_level = None
            seed = None

            security_level_bytes = None
            key_bytes = None
        elif bytes(msg)[0] == 0x50 and len(msg) == 2:
            logger.debug('diag session confirmed')
            diag_level = diag_level_bytes
        elif bytes(msg)[0] == 0x67 and len(msg) == 4:
            seed_bytes = bytes(msg)[2:6]
            security_level_bytes = bytes(msg)[1:2]
            logger.debug('seed seen: ' + seed_bytes.hex() + ' (level ' + security_level_bytes.hex() + ')')
            seed = seed_bytes
            security_level = security_level_bytes
        elif bytes(msg)[0] == 0x27 and len(msg) == 4:
            if security_level_bytes is None:
                logger.error('ERROR msg out of sequence (security level not yet confirmed): ' + bytes(msg).hex())
                continue

            if bytes(msg)[1] == security_level_bytes[0] + 1:
                key_bytes = bytes(msg)[2:6]
                logger.debug('key  seen: ' + key_bytes.hex())
            else:
                logger.error('ERROR msg out of sequence (security level reply mismatch): ' + bytes(msg).hex())
        elif bytes(msg)[0] == 0x67 and len(msg) < 4:
            if security_level_bytes is None:
                logger.error('ERROR msg out of sequence (security level not yet confirmed): ' + bytes(msg).hex())
                continue

            if bytes(msg)[1] == security_level_bytes[0] + 1:
                logger.info('key confirmed')
                key = key_bytes
                row = pd.DataFrame({'diagnostic level (hex)': [diag_level.hex()],
                                    'security level (hex)': [security_level.hex()],
                                    'seed (hex)': [seed.hex()],
                                    'key (hex)': [key.hex()]
                                    })
                df = pd.concat([df, row], ignore_index=True)
            else:
                logger.error('ERROR msg out of sequence (security level reply mismatch): ' + bytes(msg).hex())

    return df


from scapy.contrib.isotp.isotp_utils import ISOTPSession

df = None


@interact_manual
def logread(
        pcap_filename="your_pcap_or_pcapng_file_path_here.pcapng"):
    global df
    with PcapReader(filename=pcap_filename) as csock:
        iso_msgs = sniff(opened_socket=csock,
                         session=ISOTPSession(use_ext_address=False))
        msgs = iso_msgs

    df = dataframe_of_seedkey_exchanges(msgs)

    display(df)

    integer_df = df.map(lambda x: int(x, 16))

    p = figure(title="Seeds vs Keys (challenge response pairs)", x_axis_label='seed (challenge)',
               y_axis_label='key (response)')
    p.scatter(x='seed (hex)', y='key (hex)', source=integer_df, legend_label='challenge response pairs')
    show(p)


If you are (very) lucky you see a linear relationship and you can 'fit' it to get a closed form for your seed-key routine 🥳

If not, a reasonably simple next test is to see if your seed key routine is using XOR with a static key); we will do XOR of each pair (using `DataFrame`) magic:

In [ ]:
@interact_manual
def doitdoitnow():
    global df
    df_ints = df.map(lambda x: int(x, 16))
    (df_ints['seed (hex)'] ^ df_ints['key (hex)']).apply(hex)

If you are (very) lucky you will see a repeated number above and that means you have a static-key XOR routine. 💹